In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/test.csv


In [7]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [8]:
# read the train/test data
train_df = pd.read_csv('/kaggle/input/fake-news/train.csv')
test_df = pd.read_csv('/kaggle/input/fake-news/test.csv')

In [9]:
train_df.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
train_df.shape

(20800, 5)

In [11]:
train_df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [12]:
df=train_df.dropna()

In [13]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [14]:
X=df.drop('label', axis=1)

In [15]:
y=df['label']

In [16]:
import tensorflow as tf

In [17]:
tf.__version__

'2.13.0'

In [18]:
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


In [19]:
### Vocabulary size
voc_size=5000

In [48]:
messages=X.copy()

In [57]:
messages['title']

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                        Why the Truth Might Get You Fired
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799                            What Keeps the F-35 Alive
Name: title, Length: 18285, dtype: object

In [58]:
messages.shape

(18285, 4)

In [74]:
messages.reset_index(inplace=True)

In [76]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [77]:
import nltk
import re
from nltk.corpus import stopwords

In [78]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose

In [79]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [80]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [81]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [82]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[2138, 4114, 567, 1883, 3288, 3283, 1417, 3699, 668, 2510],
 [1030, 964, 139, 1983, 4605, 926, 85],
 [2029, 1212, 4111, 849],
 [2036, 1927, 591, 4926, 2641, 4559],
 [3422, 4605, 3947, 796, 1714, 2965, 4605, 3726, 944, 152],
 [2512,
  884,
  3911,
  4268,
  2810,
  1498,
  2143,
  2750,
  1961,
  1045,
  42,
  75,
  1654,
  4213,
  85],
 [1006, 3948, 3143, 1381, 2436, 4660, 2514, 3935, 1936, 1819, 252],
 [1829, 1867, 1914, 2836, 2610, 401, 1498, 3573, 1936, 1819, 252],
 [14, 4020, 257, 1920, 3795, 2880, 4275, 1288, 1498, 2981],
 [514, 505, 4023, 97, 2637, 1613, 2443, 1671],
 [523, 4793, 4712, 1510, 2847, 2710, 3893, 4046, 296, 545, 3010],
 [4926, 1790, 3288, 2880, 1498, 2610],
 [3100, 3858, 4557, 4955, 3075, 3921, 1717, 3418, 4756],
 [4755, 1066, 4657, 341, 2394, 2944, 2217, 1936, 1819, 252],
 [1978, 2685, 3543, 1561, 2278, 1936, 1819, 252],
 [2987, 2260, 3701, 2905, 3866, 4990, 3530, 2926, 2475, 4201],
 [1828, 1512, 964],
 [3141, 2910, 644, 3576, 1498, 4117, 1855, 85],
 [1687, 4671, 1

In [83]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [84]:
onehot_repr[1]

[1030, 964, 139, 1983, 4605, 926, 85]

In [85]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[2138 4114  567 ...    0    0    0]
 [1030  964  139 ...    0    0    0]
 [2029 1212 4111 ...    0    0    0]
 ...
 [2477  780 1386 ...    0    0    0]
 [ 105 2610 2920 ...    0    0    0]
 [4125 2024 1147 ...    0    0    0]]


In [86]:
embedded_docs[1]

array([1030,  964,  139, 1983, 4605,  926,   85,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [87]:
embedded_docs[0]

array([2138, 4114,  567, 1883, 3288, 3283, 1417, 3699,  668, 2510,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)